In [5]:
import os

import pandas as pd
import requests
from dotenv import load_dotenv

In [3]:
load_dotenv()
url = os.getenv('URL')
url

'https://remotive.com/api/remote-jobs'

In [ ]:
response = requests.get(url)
VALID_STATUS = 200
if response.status_code == VALID_STATUS:
    data = response.json()['jobs']
else:
    print(f'Error: {response.status_code}')

In [44]:
df = pd.DataFrame.from_dict(data)
df = df[
    [
        'id',
        'category',
        'tags',
        'title',
        'company_name',
        'job_type',
        'candidate_required_location',
        'salary',
        'url',
        'publication_date',
    ]
]
df.loc[df["salary"] == "", "salary"] = "N/A"
df['publication_date'] = pd.to_datetime(df['publication_date']).dt.date
df = df.explode("tags", ignore_index=True).reset_index(drop=True)
df

,id,category,tags,title,company_name,job_type,candidate_required_location,salary,url,publication_date
0,2018366,Design,CSS,AI-Enhanced Design Lead,Cognits,contract,Guatemala,N/A,https://remotive.com/remote-jobs/design/ai-enh...,2025-06-14
1,2018366,Design,html,AI-Enhanced Design Lead,Cognits,contract,Guatemala,N/A,https://remotive.com/remote-jobs/design/ai-enh...,2025-06-14
2,2018366,Design,javascript,AI-Enhanced Design Lead,Cognits,contract,Guatemala,N/A,https://remotive.com/remote-jobs/design/ai-enh...,2025-06-14
3,2018366,Design,UI/UX,AI-Enhanced Design Lead,Cognits,contract,Guatemala,N/A,https://remotive.com/remote-jobs/design/ai-enh...,2025-06-14
4,2018366,Design,AI/ML,AI-Enhanced Design Lead,Cognits,contract,Guatemala,N/A,https://remotive.com/remote-jobs/design/ai-enh...,2025-06-14
...,...,...,...,...,...,...,...,...,...,...
7005,1986422,Sales / Business,product development,Account Director,StackAdapt,full_time,USA,highly competitive salary + commission structure,https://remotive.com/remote-jobs/sales-busines...,2025-03-19
7006,1986422,Sales / Business,advertising,Account Director,StackAdapt,full_time,USA,highly competitive salary + commission structure,https://remotive.com/remote-jobs/sales-busines...,2025-03-19
7007,1986422,Sales / Business,onboarding,Account Director,StackAdapt,full_time,USA,highly competitive salary + commission structure,https://remotive.com/remote-jobs/sales-busines...,2025-03-19
7008,1986422,Sales / Business,travel,Account Director,StackAdapt,full_time,USA,highly competitive salary + commission structure,https://remotive.com/remote-jobs/sales-busines...,2025-03-19


In [100]:
df_python = df.loc[df["tags"] == "python", "id"].to_frame()
df_merged = pd.merge(df, df_python, on="id", how="inner")
df_merged = df_merged.loc[df_merged["tags"] != "python"]
df_grouped = df_merged.groupby("tags").size().reset_index(name="count").sort_values(["count", "tags"], ascending=[False, True]).head(5)
df_grouped

,tags,count
192,sql,86
3,AWS,64
138,kubernetes,47
11,CI/CD,36
202,terraform,35


In [101]:
df_python = df.loc[df["tags"] == "python", "id"].to_frame()
df_merged = pd.merge(df, df_python, on="id", how="inner")
list_grouped = df_merged[df_merged["tags"] != "python"]["tags"].value_counts()#.head(5).index.to_list()
list_grouped

tags
sql                 86
AWS                 64
kubernetes          47
CI/CD               36
terraform           35
                    ..
IaaS                 1
product security     1
enterprise sales     1
tls                  1
debugging            1
Name: count, Length: 221, dtype: int64